In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [4]:
from crewai import Agent,Task,Crew,LLM
from dotenv import  load_dotenv
import os
import json
import yaml
load_dotenv()

api_key = os.getenv("GEMINI_API_KEY")
serper_api_key=os.getenv("SERPER_API_KEY")

os.environ["OPENAI_API_KEY"] = "dummy"

llm = LLM(model="gemini/gemini-2.5-flash",
          provider="gemini",
          temperature=0.7,
          api_key=api_key)

In [5]:
# Define file paths for YAML configurations
files = {
    'lead_agents': 'config/lead_qualification_agents.yaml',
    'lead_tasks': 'config/lead_qualification_tasks.yaml',
    'email_agents': 'config/email_engagement_agents.yaml',
    'email_tasks': 'config/email_engagement_tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
lead_agents_config = configs['lead_agents']
lead_tasks_config = configs['lead_tasks']
email_agents_config = configs['email_agents']
email_tasks_config = configs['email_tasks']

In [6]:
from pydantic import BaseModel, Field
from typing import Dict, Optional, List, Set, Tuple

class LeadPersonalInfo(BaseModel):
    name: str = Field(..., description="The full name of the lead.")
    job_title: str = Field(..., description="The job title of the lead.")
    role_relevance: int = Field(..., ge=0, le=10, description="A score representing how relevant the lead's role is to the decision-making process (0-10).")
    professional_background: Optional[str] = Field(..., description="A brief description of the lead's professional background.")

class CompanyInfo(BaseModel):
    company_name: str = Field(..., description="The name of the company the lead works for.")
    industry: str = Field(..., description="The industry in which the company operates.")
    company_size: int = Field(..., description="The size of the company in terms of employee count.")
    revenue: Optional[float] = Field(None, description="The annual revenue of the company, if available.")
    market_presence: int = Field(..., ge=0, le=10, description="A score representing the company's market presence (0-10).")

class LeadScore(BaseModel):
    score: int = Field(..., ge=0, le=100, description="The final score assigned to the lead (0-100).")
    scoring_criteria: List[str] = Field(..., description="The criteria used to determine the lead's score.")
    validation_notes: Optional[str] = Field(None, description="Any notes regarding the validation of the lead score.")

class LeadScoringResult(BaseModel):
    personal_info: LeadPersonalInfo = Field(..., description="Personal information about the lead.")
    company_info: CompanyInfo = Field(..., description="Information about the lead's company.")
    lead_score: LeadScore = Field(..., description="The calculated score and related information for the lead.")

In [9]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

## Lead Qualification Crew, Agents and Tasks

In [10]:
# Creating Agents
lead_data_agent = Agent(
  config=lead_agents_config['lead_data_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()],
    llm=llm,
)

cultural_fit_agent = Agent(
  config=lead_agents_config['cultural_fit_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()],
    llm=llm,
)

scoring_validation_agent = Agent(
  config=lead_agents_config['scoring_validation_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()],
    llm=llm,
)

# Creating Tasks
lead_data_task = Task(
  config=lead_tasks_config['lead_data_collection'],
  agent=lead_data_agent
)

cultural_fit_task = Task(
  config=lead_tasks_config['cultural_fit_analysis'],
  agent=cultural_fit_agent
)

scoring_validation_task = Task(
  config=lead_tasks_config['lead_scoring_and_validation'],
  agent=scoring_validation_agent,
  context=[lead_data_task, cultural_fit_task],
  output_pydantic=LeadScoringResult
)

# Creating Crew
lead_scoring_crew = Crew(
  agents=[
    lead_data_agent,
    cultural_fit_agent,
    scoring_validation_agent
  ],
  tasks=[
    lead_data_task,
    cultural_fit_task,
    scoring_validation_task
  ],
  verbose=True,
    manager_llm=llm
)

## Email Engagement Crew

In [12]:
# Creating Agents
email_content_specialist = Agent(
  config=email_agents_config['email_content_specialist'],
    llm=llm,
)

engagement_strategist = Agent(
  config=email_agents_config['engagement_strategist'],
    llm=llm,
)

# Creating Tasks
email_drafting = Task(
  config=email_tasks_config['email_drafting'],
  agent=email_content_specialist
)

engagement_optimization = Task(
  config=email_tasks_config['engagement_optimization'],
  agent=engagement_strategist
)

# Creating Crew
email_writing_crew = Crew(
  agents=[
    email_content_specialist,
    engagement_strategist
  ],
  tasks=[
    email_drafting,
    engagement_optimization
  ],
  verbose=True,
    manager_llm=llm
)

## Creating Complete Sales Flow

In [13]:
from crewai import Flow
from crewai.flow.flow import listen, start

class SalesPipeline(Flow):
    @start()
    def fetch_leads(self):
        # Pull our leads from the database
        leads = [
            {
                "lead_data": {
                    "name": "João Moura",
                    "job_title": "Director of Engineering",
                    "company": "Clearbit",
                    "email": "joao@clearbit.com",
                    "use_case": "Using AI Agent to do better data enrichment."
                },
            },
        ]
        return leads

    @listen(fetch_leads)
    def score_leads(self, leads):
        scores = lead_scoring_crew.kickoff_for_each(leads)
        self.state["score_crews_results"] = scores
        return scores

    @listen(score_leads)
    def store_leads_score(self, scores):
        # Here we would store the scores in the database
        return scores

    @listen(score_leads)
    def filter_leads(self, scores):
        return [score for score in scores if score['lead_score'].score > 70]

    @listen(filter_leads)
    def write_email(self, leads):
        scored_leads = [lead.to_dict() for lead in leads]
        emails = email_writing_crew.kickoff_for_each(scored_leads)
        return emails

    @listen(write_email)
    def send_email(self, emails):
        # Here we would send the emails to the leads
        return emails

flow = SalesPipeline()

╭──────────────────────────────────────────────── Flow Execution ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Starting Flow Execution                                                                                        │
│  Name: SalesPipeline                                                                                            │
│  ID: 1f9064a8-49f6-4373-bf25-6be1c6acb73e                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [14]:
flow.plot()

Plot saved as crewai_flow.html


In [16]:
from IPython.display import IFrame

IFrame(src='./crewai_flow.html', width='150%', height=600)

In [18]:
emails = await flow.kickoff_async()

 Flow started with ID: 1f9064a8-49f6-4373-bf25-6be1c6acb73e

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: a761c4a9-b8bb-4f0f-9e7e-62cb146e36f0                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Lead Data Specialist                                                                                    │
│                                                                                                                 │
│  Task: Collect and analyze the following information about the lead:                                            │
│  - Personal Information:                                                                                        │
│    - Name: Obtain the full name of the lead.                                                                    │
│    - Job Title: Determine the lead's current job title.                                                         │
│    - Role Relevance: Assess how relevant the lead's role is to the decision-making process on a scale from 0    │
│  to 10.                                                                                                         │
│    - Professional Background: Optionally, gather a brief description of the lead's professional background.     │
│                                                                                                                 │
│  - Company Information:                                                                                         │
│    - Company Name: Identify the name of the company the lead works for.                                         │
│    - Industry: Determine the industry in which the company operates.                                            │
│    - Company Size: Estimate the size of the company in terms of employee count.                                 │
│    - Revenue: If available, collect information on the annual revenue of the company.                           │
│    - Market Presence: Evaluate the company's market presence on a scale from 0 to 10.                           │
│                                                                                                                 │
│  - Our Company and Product:                                                                                     │
│    - Company Name: CrewAI                                                                                       │
│    - Product: Multi-Agent Orchestration Platform                                                                │
│    - ICP: Enterprise companies looking into Agentic automation.                                                 │
│    - Pitch: We are a platform that allows you to orchestrate AI Agents for automations to any vertical.         │
│                                                                                                                 │
│  -Lead Data:                                                                                                    │
│    {'name': 'João Moura', 'job_title': 'Director of Engineering', 'company': 'Clearbit', 'email':               │
│  'joao@clearbit.com', 'use_case': 'Using AI Agent to do better data enrichment.'}                               │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Lead Data Specialist                                                                                    │
│                                                                                                                 │
│  Thought: Action: Search the internet with Serper                                                               │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"search_query\": \"Jo\\u00e3o Moura Director of Engineering Clearbit LinkedIn\"}"                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Engagement Optimization Specialist                                                                      │
│                                                                                                                 │
│  Task: Review the personalized email draft and optimize it with strong CTAs and engagement hooks. Keep in mind  │
│  they reached out and filled a lead form. Keep it short and to the point. Don't use any salutations or closing  │
│  remarks, nor too complex sentences. Ensure the email encourages the lead to schedule a meeting or take         │
│  another desired action immediately.                                                                            │
│  Our Company and Product: - Company Name: CrewAI - Product: Multi-Agent Orchestration Platform - ICP:           │
│  Enterprise companies looking into Agentic automation. - Pitch: We are a platform that allows you to            │
│  orchestrate AI Agents for automations to any vertical.                                                         │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Engagement Optimization Specialist                                                                      │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  You're looking into agentic automation for your enterprise. CrewAI delivers multi-agent orchestration.         │
│  Our platform empowers you to deploy AI Agents, automating workflows across any vertical.                       │
│  Want to unlock immediate operational efficiencies?                                                             │
│  **Schedule a 15-minute solution walkthrough now.**                                                             │
│  See how CrewAI can integrate seamlessly and transform your operations.                                         │
│  **Review our enterprise capabilities.**                                                                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: b0c3c14d-bb6c-4fa0-abf1-271a4510c9e8                                                                     │
│  Agent: Engagement Optimization Specialist                                                                      │
│                                                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 59737967-447f-4e63-a473-73f1974661a1                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: You're looking into agentic automation for your enterprise. CrewAI delivers multi-agent          │
│  orchestration.                                                                                                 │
│  Our platform empowers you to deploy AI Agents, automating workflows across any vertical.                       │
│  Want to unlock immediate operational efficiencies?                                                             │
│  **Schedule a 15-minute solution walkthrough now.**                                                             │
│  See how CrewAI can integrate seamlessly and transform your operations.                                         │
│  **Review our enterprise capabilities.**                                                                        │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Flow Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Flow Execution Completed                                                                                       │
│  Name: SalesPipeline                                                                                            │
│  ID: 1f9064a8-49f6-4373-bf25-6be1c6acb73e                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [19]:
import pandas as pd

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([flow.state["score_crews_results"][0].token_usage.dict()])

# Calculate total costs
costs = 0.150 * df_usage_metrics['total_tokens'].sum() / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Display the DataFrame
df_usage_metrics

Total costs: $0.0041


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,27100,19284,5089,7816,8


In [20]:
import pandas as pd

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([emails[0].token_usage.dict()])

# Calculate total costs
costs = 0.150 * df_usage_metrics['total_tokens'].sum() / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Display the DataFrame
df_usage_metrics

Total costs: $0.0005


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,3366,1034,0,2332,2


In [21]:
scores = flow.state["score_crews_results"]

In [22]:
import pandas as pd
from IPython.display import display, HTML

lead_scoring_result = scores[0].pydantic

# Create a dictionary with the nested structure flattened
data = {
    'Name': lead_scoring_result.personal_info.name,
    'Job Title': lead_scoring_result.personal_info.job_title,
    'Role Relevance': lead_scoring_result.personal_info.role_relevance,
    'Professional Background': lead_scoring_result.personal_info.professional_background,
    'Company Name': lead_scoring_result.company_info.company_name,
    'Industry': lead_scoring_result.company_info.industry,
    'Company Size': lead_scoring_result.company_info.company_size,
    'Revenue': lead_scoring_result.company_info.revenue,
    'Market Presence': lead_scoring_result.company_info.market_presence,
    'Lead Score': lead_scoring_result.lead_score.score,
    'Scoring Criteria': ', '.join(lead_scoring_result.lead_score.scoring_criteria),
    'Validation Notes': lead_scoring_result.lead_score.validation_notes
}

# Convert the dictionary to a DataFrame
df = pd.DataFrame.from_dict(data, orient='index', columns=['Value'])

# Reset the index to turn the original column names into a regular column
df = df.reset_index()

# Rename the index column to 'Attribute'
df = df.rename(columns={'index': 'Attribute'})

# Create HTML table with bold attributes and left-aligned values
html_table = df.style.set_properties(**{'text-align': 'left'}) \
                     .format({'Attribute': lambda x: f'<b>{x}</b>'}) \
                     .hide(axis='index') \
                     .to_html()

# Display the styled HTML table
display(HTML(html_table))

Attribute,Value
Name,João Moura
Job Title,Director of AI Engineering
Role Relevance,9
Professional Background,"João Moura served as the Director of AI Engineering at Clearbit, where he was responsible for leading their enterprise AI products. He possesses close to 20 years of experience in software and AI leadership and played a key role in navigating Clearbit's AI initiatives through its acquisition by HubSpot."
Company Name,Clearbit
Industry,"B2B Marketing Intelligence, Data Enrichment, Data & Analytics, Marketing Technology"
Company Size,110
Revenue,31.500000
Market Presence,8
Lead Score,85


In [23]:
import textwrap

result_text = emails[0].raw
wrapped_text = textwrap.fill(result_text, width=80)
print(wrapped_text)

You're looking into agentic automation for your enterprise. CrewAI delivers
multi-agent orchestration. Our platform empowers you to deploy AI Agents,
automating workflows across any vertical. Want to unlock immediate operational
efficiencies? **Schedule a 15-minute solution walkthrough now.** See how CrewAI
can integrate seamlessly and transform your operations. **Review our enterprise
capabilities.**


In [25]:
from crewai import Flow
from crewai.flow.flow import listen, start, and_, or_, router

class SalesPipeline(Flow):

  @start()
  def fetch_leads(self):
    # Pull our leads from the database
    # This is a mock, in a real-world scenario, this is where you would
    # fetch leads from a database
    leads = [
      {
        "lead_data": {
          "name": "João Moura",
          "job_title": "Director of Engineering",
          "company": "Clearbit",
          "email": "joao@clearbit.com",
          "use_case": "Using AI Agent to do better data enrichment."
        },
      },
    ]
    return leads

  @listen(fetch_leads)
  def score_leads(self, leads):
    scores = lead_scoring_crew.kickoff_for_each(leads)
    self.state["score_crews_results"] = scores
    return scores

  @listen(score_leads)
  def store_leads_score(self, scores):
    # Here we would store the scores in the database
    return scores

  @listen(score_leads)
  def filter_leads(self, scores):
    return [score for score in scores if score['lead_score'].score > 70]

  @listen(and_(filter_leads, store_leads_score))
  def log_leads(self, leads):
    print(f"Leads: {leads}")

  @router(filter_leads)
  def count_leads(self, scores):
    if len(scores) > 10:
      return 'high'
    elif len(scores) > 5:
      return 'medium'
    else:
      return 'low'

  @listen('high')
  def store_in_salesforce(self, leads):
    return leads

  @listen('medium')
  def send_to_sales_team(self, leads):
    return leads

  @listen('low')
  def write_email(self, leads):
    scored_leads = [lead.to_dict() for lead in leads]
    emails = email_writing_crew.kickoff_for_each(scored_leads)
    return emails

  @listen(write_email)
  def send_email(self, emails):
    # Here we would send the emails to the leads
    return emails

In [29]:
flow = SalesPipeline()
flow.plot('crewai_flow_complex')

╭──────────────────────────────────────────────── Flow Execution ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Starting Flow Execution                                                                                        │
│  Name: SalesPipeline                                                                                            │
│  ID: 5cb37c9d-4488-4133-98a2-9023da861ad1                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Plot saved as crewai_flow_complex.html


In [31]:
from IPython.display import IFrame

IFrame(src='./crewai_flow_complex.html', width='150%', height=600)